In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pyvista as pv
import pandas as pd
from scipy import interpolate
import CollisionModule

In [ ]:
mu, mu2,  sigma = -0.03,0.03, 0.010 # mean and standard deviation

N = int(1e7)
s = np.random.normal(mu, sigma, N)
s2 = np.random.normal(mu2, sigma, N)

print(s.shape)
s = np.concatenate((s, s2))
plt.hist(s, 200, density=True)

plt.show()

In [ ]:
gen_dist_x = s
gen_dist_y = np.random.uniform(-0.1, 0.1, N*2)

xbins = np.linspace(-0.1,0.1,200)
ybins = np.linspace(-0.1,0.1,200)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(gen_dist_x, bins=xbins)
y_bins = np.histogram(gen_dist_y, bins=ybins)

H, xedges, yedges = np.histogram2d(gen_dist_x, gen_dist_y, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('Position of released Al particles on target ')
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
np.save('velosity_matrix_2e7_SMD_4000K', velosity_matrix)

In [ ]:
np.save('release_pos_2e7_SMD_3',release_pos)
np.save('velosity_matrix_2e7_SMD_3', velosity_matrix)

In [ ]:
def rfunc(x, n, beta): #Release factor function
    # print("-------rfunc------")
    # print(x)
    # n = 1.6
    # beta = -0.7
    y = np.cos(x) ** n * (1 + beta * np.cos(x) ** 2)# * (n ** 2 + 4 * n + 3) / (n * beta + n + beta + 3) /2 / pi
    return y

In [ ]:
def rfunc2(x, n, beta): #Release factor function
    # print("-------rfunc------")
    # print(x)
    # n = 1.6
    # beta = -0.7
    y = np.cos(x) ** n * (1 + beta * np.cos(x) ** 2) * (n ** 2 + 4 * n + 3) / (n * beta + n + beta + 3) /2 / np.pi
    return y

In [ ]:
theta_bins_size = 100
theta_bins = np.linspace(-np.pi/2, np.pi/2, theta_bins_size)
theta_hist_x = theta_bins + np.pi/((theta_bins_size-1)*2)
theta_hist_x = theta_hist_x[:-1]

theta_hist_y = rfunc2(theta_hist_x, 1.6, -0.7)

fig, ax = plt.subplots(1, 1)

ax.plot(theta_hist_x, theta_hist_y)

In [ ]:
theta_sample = np.array([])
theta_bins = np.linspace(-np.pi/2, np.pi/2, theta_bins_size)
theta_hist_x = theta_bins + np.pi/((theta_bins_size-1)*2)
theta_hist_x = theta_hist_x[:-1]

for i in range(theta_bins.shape[0] - 1):
    theta_sample = np.concatenate(( theta_sample, np.random.uniform(theta_bins[i], theta_bins[i+1],\
                                 int(rfunc2(theta_hist_x[i], 1.6, -0.7)/np.sum(rfunc2(theta_hist_x, 1.6, -0.7))*1e7))))

plt.hist(theta_sample, bins=100)

In [ ]:
theta_sample = np.array([])
theta_hist_y *= 1.1e6
for i in range(theta_bins.shape[0] - 1):
    theta_sample = np.concatenate(( theta_sample, np.random.uniform(theta_bins[i], theta_bins[i+1], int(theta_hist_y[i]))))

plt.hist(theta_sample, bins=100)

In [ ]:
theta_sample.shape

In [ ]:
np.random.shuffle(theta_sample)
plt.hist(theta_sample[:int(gen_dist_x.shape[0])], bins=100)

In [ ]:
def Energy_theta_func(C, enery, theta):
    U = 3.36 #binding
    m = 0.212
    Alpha = 0.14
    n = 0.5
    A = 13
    # m_t = 44.803928e-27
    # m_i = 66.335e-27
    m_t = 27
    m_i = 40
    q = 2 - m_t/(4*m_i)
    Q = 0.55
    E_inc = 500 # eVsb Vsb self bias
    if theta < np.pi/4 or theta >= -np.pi/4:
        return C * enery/(enery + U)**(3-2*m)*(1 - ((U + enery)/(U + Alpha*E_inc))**n)*np.exp(-A*((m_i/m_t)*((U + enery * np.cos(theta)**q)/E_inc))**Q)
    else:
        return C * enery/(enery + U)**(3-2*m)*(1 - ((U + enery)/(U + Alpha*E_inc))**n)*np.exp(-A*((m_i/m_t)*((U + enery * (1-np.sin(theta*2)/2)**q)/E_inc))**Q)

In [ ]:
theta = 0.15*np.pi
enery = np.linspace(0, 100, 100)

fig, ax = plt.subplots(1,1)
ax.plot(enery, Energy_theta_func(10, enery, theta))
# ax.set_yscale('log')

In [ ]:
enery_bins = np.linspace(0, 50, 100)

energy_theta_tuple = []
for j in range(theta_hist_x.shape[0]):
    print(j)
    enery_sample = np.array([])
    for i in range(enery_bins.shape[0] - 1):
        enery_sample = np.concatenate(( enery_sample, \
                                    np.random.uniform(enery_bins[i], enery_bins[i+1], \
                                    int(Energy_theta_func(10, enery_bins[i], theta_hist_x[j])/np.sum(Energy_theta_func(10, enery_bins, theta_hist_x[j]))*1e6))))
    energy_theta_tuple.append(enery_sample)

for i in range(len(energy_theta_tuple)):
    np.random.shuffle(energy_theta_tuple[i])

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Energy distribution of sputtered aluminum")
ax.hist(energy_theta_tuple[0],density=True, bins=100,histtype='step', stacked=True, fill=False, label='0degree')
ax.hist(energy_theta_tuple[25],density=True, bins=100,histtype='step', stacked=True, fill=False, label='45degree')
ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Energy distribution of sputtered aluminum")
ax.hist(energy_theta_tuple[0],density=True, bins=100,histtype='step', stacked=True, fill=False, label='90degree')
ax.hist(energy_theta_tuple[25],density=True, bins=100,histtype='step', stacked=True, fill=False, label='45degree')
ax.hist(energy_theta_tuple[50],density=True, bins=100,histtype='step', stacked=True, fill=False, label='0degree')
ax.hist(energy_theta_tuple[98],density=True, bins=100,histtype='step', stacked=True, fill=False, label='-90degree')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
test_sample = theta_sample[:int(gen_dist_x.shape[0])]
theta_hist_sample = np.histogram(test_sample, bins=99)

energy_sample = np.array([])
energymin = []
for i in range(99):
    energy_theta_sample = energy_theta_tuple[i][:theta_hist_sample[0][i] ]
    # print(energy_theta_sample.shape)
    # print(theta_hist_sample[0][i])
    if energy_theta_sample.shape[0] != theta_hist_sample[0][i]:
        print('wrong')
    energy_sample = np.concatenate((energy_sample, energy_theta_sample))
    # print(np.average(energy_theta_sample))
    energymin.append(np.average(energy_theta_sample))

print(energy_sample.shape)
print(gen_dist_x.shape[0])

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(theta_bins[1:], energymin)

In [ ]:
theta_hist_sample[0]

In [ ]:
ev = 1.602176634e-19
velosity_norm_energy = np.sqrt((energy_sample*ev)*2/44.803928e-27)

test_sample_sort = np.sort(test_sample)

theta = test_sample
phi = np.random.rand(gen_dist_x.shape[0])*2*np.pi
print(phi.shape)
print(theta.shape)
# vel_x = np.cos(phi)*np.sin(theta)*velosity_norm_energy
# vel_y = np.sin(phi)*np.sin(theta)*velosity_norm_energy
random2 = np.random.rand(gen_dist_x.shape[0])
vel_x = np.sin(theta)*(np.cos(2*np.pi*random2))
vel_y = np.sin(theta)*(np.sin(2*np.pi*random2))

vel_z = np.cos(theta)

velosity_matrix = np.array([vel_x, vel_y, -vel_z]).T

rng = np.random.default_rng()
velosity_matrix = rng.permuted(velosity_matrix, axis=0)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(energy_sample, bins=100,histtype='step', stacked=True, fill=False, label='e')

# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(theta_sample, bins=100,histtype='step', stacked=True, fill=False, label='e')

# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
1.66e-27*27

In [ ]:
ev = 1.602176634e-19
velosity_norm_energy = np.sqrt((energy_sample*ev)*2/4.4803928e-26)

test_sample_sort = np.sort(test_sample)

theta = test_sample
phi = np.random.rand(gen_dist_x.shape[0])*2*np.pi
print(phi.shape)
print(theta.shape)
# vel_x = np.cos(phi)*np.sin(theta)*velosity_norm_energy
# vel_y = np.sin(phi)*np.sin(theta)*velosity_norm_energy
random2 = np.random.rand(gen_dist_x.shape[0])
vel_x = np.sin(theta)*(np.cos(2*np.pi*random2))*velosity_norm_energy
vel_y = np.sin(theta)*(np.sin(2*np.pi*random2))*velosity_norm_energy

vel_z = np.cos(theta)*velosity_norm_energy

velosity_matrix = np.array([vel_x, vel_y, -vel_z]).T

rng = np.random.default_rng()
velosity_matrix = rng.permuted(velosity_matrix, axis=0)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
release_pos = np.array([gen_dist_x, gen_dist_y, np.zeros_like(gen_dist_x)]).T

print(release_pos.shape)
print(velosity_matrix.shape)

In [ ]:
np.save('release_pos_2e7_SMD_2',release_pos)
np.save('velosity_matrix_2e7_SMD_2', velosity_matrix)

In [ ]:
ev = 1.602176634e-19
velosity_norm_energy = np.sqrt((energy_sample*ev)*2/44.803928e-27)

test_sample_sort = np.sort(test_sample)

theta = test_sample
phi = np.random.rand(gen_dist_x.shape[0])*2*np.pi
print(phi.shape)
print(theta.shape)
# vel_x = np.cos(phi)*np.sin(theta)*velosity_norm_energy
# vel_y = np.sin(phi)*np.sin(theta)*velosity_norm_energy
random2 = np.random.rand(gen_dist_x.shape[0])
vel_x = np.sin(theta)*(np.cos(2*np.pi*random2))*velosity_norm_energy
vel_y = np.sin(theta)*(np.sin(2*np.pi*random2))*velosity_norm_energy

vel_z = np.cos(theta)*velosity_norm_energy

velosity_matrix = np.array([vel_x, vel_y, -vel_z]).T

rng = np.random.default_rng()
velosity_matrix = rng.permuted(velosity_matrix, axis=0)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
ev = 1.602176634e-19
velosity_norm_energy = np.sqrt((energy_sample*ev)*2/44.803928e-27)

test_sample_sort = np.sort(test_sample)

theta = test_sample
phi = np.random.rand(gen_dist_x.shape[0])*2*np.pi
# vel_x = np.cos(phi)*np.sin(theta)*velosity_norm_energy
# vel_y = np.sin(phi)*np.sin(theta)*velosity_norm_energy
random2 = np.random.rand(gen_dist_x.shape[0])
vel_x = np.sin(theta)*(np.cos(2*np.pi*random2))*velosity_norm_energy
vel_y = np.sin(theta)*(np.sin(2*np.pi*random2))*velosity_norm_energy

vel_z = np.cos(theta)*velosity_norm_energy

velosity_matrix = np.array([vel_x, vel_y, -vel_z]).T

rng = np.random.default_rng()
velosity_matrix = rng.permuted(velosity_matrix, axis=0)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_norm_energy, bins=100,histtype='step', stacked=True, fill=False, label='vel_x')

# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
from scipy.stats import rv_continuous
from scipy.stats.sampling import NumericalInversePolynomial
from scipy.special import gamma, factorial

In [ ]:
def diVr_func(d_refi, eVr, wi):
    kb = 1.380649e-23
    Tref = 650
    ev = 1.60217663e-19
    diVr = d_refi * np.sqrt(((kb*Tref)/(eVr*ev))**(wi-1/2)*gamma(5/2 - wi))
    return diVr

In [ ]:
def DXsec(d_refi, eVr, wi, chi):
    if chi == 0:
        return 0
    if chi != 0:
        return diVr_func(d_refi, eVr, wi)**2/(np.pi * np.sin(chi))*(1 - chi/np.pi)

In [ ]:
def TotXsec(d_refi, eVr, wi):
    return np.pi * diVr_func(d_refi, eVr, wi)**2

In [ ]:
def DXsec(d_refi, eVr, wi, chi):
    return diVr_func(d_refi, eVr, wi)**2/(np.pi * np.sin(chi))*(1 - chi/np.pi)

In [ ]:
enery = np.linspace(1, 100, 100)

fig, ax = plt.subplots(1,1)
ax.plot(enery, TotXsec((4.614 + 4.151)/2*1e-10, enery, 0.721))

In [ ]:
enery = np.linspace(0, 100, 100)

fig, ax = plt.subplots(1,1)
ax.plot(enery, TotXsec((4.614 + 4.151)/2*1e-10, enery, 0.721))

In [ ]:
chiX = np.linspace(0.000001, np.pi*0.999999999, 100)

fig, ax = plt.subplots(1,1)
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 100, 0.721, chiX), label='100eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 10, 0.721, chiX), label='10eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 1, 0.721, chiX), label='1eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 0.1, 0.721, chiX), label='0.1eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 0.01, 0.721, chiX), label='0.01eV')
ax.set_xlabel('$\chi(rad)$')
ax.set_xlim(0, np.pi)
ax.set_ylim(1e-17, 1e-13)
ax.set_ylabel('$\sigma(cm^2 /rad)$')
ax.set_yscale('log')
ax.legend()

In [ ]:
chiX = np.linspace(0.01, np.pi*0.999999999, 100)

fig, ax = plt.subplots(1,1)
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 100, 0.721, chiX), label='100eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 10, 0.721, chiX), label='10eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 1, 0.721, chiX), label='1eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 0.1, 0.721, chiX), label='0.1eV')
ax.plot(chiX, DXsec((4.614 + 4.151)/2*1e-8, 0.01, 0.721, chiX), label='0.01eV')
ax.set_xlabel('$\chi(rad)$')
ax.set_xlim(0, np.pi)
# ax.set_ylim(1e-17, 1e-13)
ax.set_ylabel('$\sigma(cm^2 /rad)$')
# ax.set_yscale('log')
ax.legend()

In [ ]:
DX100 = DXsec((4.614 + 4.151)/2*1e-8, 100, 0.721, chiX)

DX100_sample = np.array([])
for i in range(chiX.shape[0] - 1):
    DX100_sample = np.concatenate(( DX100_sample, np.random.uniform(chiX[i], chiX[i+1], int(DX100[i]/np.sum(DX100)*1e6))))


In [ ]:
DX100_sample.sum()

In [ ]:
plt.hist(DX100_sample, bins= 100, histtype='step', stacked=True, fill=False, label='rvs50')
plt.legend()

In [ ]:
energy = np.linspace(0.1, 50, 50)
chiX = np.linspace(0.01, np.pi*0.999999999, 100)

DX_tuple = []
for j in range(energy.shape[0]):
    print(j)
    DX_sample = np.array([])
    DX_sample = np.array([])
    for i in range(chiX.shape[0] - 1):
        DX_sample = np.concatenate(( DX_sample, \
                                    np.random.uniform(chiX[i], chiX[i+1], \
                                    int(DXsec((4.614 + 4.151)/2*1e-8, energy[j], 0.721, chiX[i])*1e19))))
    DX_tuple.append(DX_sample)

for i in range(len(DX_tuple)):
    np.random.shuffle(DX_tuple[i])

In [ ]:
plt.hist(DX_tuple[20], bins= 100, histtype='step', stacked=True, fill=False, label='rvs10')
plt.hist(DX_tuple[0], bins= 100, histtype='step', stacked=True, fill=False, label='rvs50')
plt.legend()

In [ ]:
class DCS_gen_50(rv_continuous):
    "Dcs distribution"
    def _pdf(self, x):
       return DXsec((4.614 + 4.151)/2*1e-10, 0.01, 0.7205, x)
    

class DCS_gen_100(rv_continuous):
    "Dcs distribution"
    def _pdf(self, x):
       return DXsec((4.614 + 4.151)/2*1e-10, 1000, 0.7205, x)
    
urng = np.random.default_rng()

dist50 =DCS_gen_50()
rng50 = NumericalInversePolynomial(dist50, domain=[0.01, np.pi], random_state=urng)
rvs50 = rng50.rvs(size=10000000)

dist100 =DCS_gen_100()
rng100 = NumericalInversePolynomial(dist100, domain=[0.01, np.pi], random_state=urng)
rvs100 = rng100.rvs(size=10000000)
print(rvs100)
plt.hist(rvs100, bins= 100, histtype='step', stacked=True, fill=False, label='rvs100')
plt.hist(rvs50, bins= 100, histtype='step', stacked=True, fill=False, label='rvs50')
plt.legend()

In [ ]:
print(rvs50.shape)

In [ ]:
transport = CollisionModule.transport(pressure_pa=0.3, timeStep= 1e-9, temperature=300, cellSize=[240, 240, 150], celllength=2, chamberSize=[0.24, 0.24], DXsec=rvs50)

In [ ]:
release_pos = np.array([gen_dist_x, gen_dist_y, np.zeros_like(gen_dist_x)]).T

print(release_pos.shape)
print(velosity_matrix.shape)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
np.save('DXsec_Al_1e8_0618', rvs100)

In [ ]:
np.save('release_pos_1e8',release_pos)
np.save('velosity_matrix_1e8', velosity_matrix)
np.save('DXsec_Al_1e8', rvs50)

In [ ]:
transport.runE(p0=release_pos, v0=velosity_matrix, time=1e-7)